In [72]:
from selenium import webdriver 
from pymongo import MongoClient
from bs4 import BeautifulSoup
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import pymongo
import requests
from time import sleep
import urllib.request
import os

In [73]:
# Pilote selenium Chrome 
browser = webdriver.Chrome()

In [74]:
#Go to the url
browser.get('https://mbasic.facebook.com/')

In [75]:
#Accept Cookies
WebDriverWait(browser, 2).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[type='submit']"))).click()

In [76]:
#target username and password
username = WebDriverWait(browser, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='email']")))
password = WebDriverWait(browser, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='pass']")))

#enter username and password
username.clear()
username.send_keys("*******")
password.clear()
password.send_keys("*******")

#target the login button and click it
login_btn = WebDriverWait(browser, 10).until(
    EC.element_to_be_clickable((By.NAME, 'login'))
)
login_btn.click()

In [ ]:
from selenium.webdriver.common.by import By
buttons = browser.find_elements(By.XPATH, '//*[contains(@class, "x1i10hfl")]')
for b in buttons:
    if "Plus tard" in b.text:
        b.click()
        break





In [120]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys

# Search
search = WebDriverWait(browser, 20).until(
    EC.element_to_be_clickable((By.XPATH, "//input[contains(@class, 'x1i10hfl')]"))
)




# Enter desired hashtag
search.clear()
search.send_keys("le décès du président Jacques Chirac ")

search.send_keys(Keys.RETURN)

In [123]:
list_dir="C:\\"
default_dir = os.path.join(list_dir,"C:/Users/yomna/Downloads/scrapper/Pictures")
opener = urllib.request.build_opener()
urllib.request.install_opener(opener)

In [121]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
import time

# 1. Trouver et cliquer sur le bouton "Publications"
try:
    # Attendre que le bouton soit cliquable (ajustez le sélecteur si nécessaire)
    publications_btn = WebDriverWait(browser, 15).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="mount_0_0_tl"]/div/div[1]/div/div[3]/div/div/div[1]/div[1]/div[1]/div/div[2]/div[1]/div[2]/div/div/div[2]/div/div[2]/div/a'))
    )
    publications_btn.click()
    print("Bouton 'Publications' cliqué avec succès.")
    time.sleep(3)  # Attendre le chargement des publications
except (NoSuchElementException, TimeoutException) as e:
    print("Échec : Bouton 'Publications' introuvable ou non cliquable.", str(e))
    exit()

Bouton 'Publications' cliqué avec succès.


## Scrapping

In [134]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import os
import json
import time

# Configuration
output_dir = "C:/Users/yomna/Downloads/scrapper/Pictures"
os.makedirs(output_dir, exist_ok=True)

def scrape_publications():
    # Attendre le chargement et faire défiler
    for _ in range(3):
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)
    
    # Sélecteur des publications
    posts = WebDriverWait(browser, 15).until(
        EC.presence_of_all_elements_located((By.XPATH, '//div[contains(@class, "xdj266r") and contains(@class, "x11i5rnm")]'))
    )
    
    result = []
    
    for post in posts:
        try:
            # Vérifier si c'est une publication sur Chirac
            post_text = post.text.lower()
            if "chirac" not in post_text and "décès" not in post_text:
                continue
            
            # Extraire le texte complet
            full_text = post.text
            
            # Extraire la première image valide
            img_url = None
            images = post.find_elements(By.TAG_NAME, 'img')
            for img in images:
                img_src = img.get_attribute('src')
                if img_src and img_src.startswith('http'):
                    img_url = img_src
                    break
            
            # Ajouter au résultat
            result.append({
                'text': full_text,
                'image': img_url if img_url else None  # None si pas d'image trouvée
            })
            
        except Exception as e:
            print(f"Erreur sur une publication : {e}")
            continue
    
    # Sauvegarde en JSON
    output_path = os.path.join(output_dir, 'publications.json')
    with open(output_path, 'w', encoding='utf-8') as f:
        json.dump(result, f, ensure_ascii=False, indent=2)
    
    print(f"Résultats sauvegardés dans {output_path}")
    print(f"Exemple de résultat : {json.dumps(result[:1], indent=2) if result else 'Aucun résultat'}")

# Exécution
scrape_publications()

Résultats sauvegardés dans C:/Users/yomna/Downloads/scrapper/Pictures\publications.json
Exemple de résultat : [
  {
    "text": "VDT ( VIE DE TAMOUL)\n  \u00b7\nSuivre\ne\no\no\nt\ns\nd\nS\np\nr\nn\n7\nl\nu\n6\nh\n6\nh\n4\n2\n0\nm\nb\n4\n4\np\n2\n2\ne\ng\n2\n7\ng\n 2\n1\ni\n1\n f\n2\nh\n5\ns\n3\nm\ni\n2\n0\nc\nr\n0\na\n9\ne\n0\n0\n3\n0\nt\ne\n  \u00b7\nD\u00e9c\u00e8s de JACQUES CHIRAC l'ancien Pr\u00e9sident de la R\u00e9publique Fran\u00e7aise \n \u00e0 86 ans.\n\u00ab L\u2019Inde, comme la Chine, est \u00e0 mes yeux une des meilleures illustrations qui soient de la permanence des grandes cultures du monde dans tout ce qu\u2019elles ont apport\u00e9 \u00e0 l\u2019humanit\u00e9 d\u2019intelligence, de connaissance et de spiritualit\u00e9. \u00bb\nToutes les r\u00e9actions :\n444\n444\n44 commentaires\n142 partages\nJ\u2019aime\nCommenter\nPartager",
    "image": "https://static.xx.fbcdn.net/images/emoji.php/v9/t74/1/16/1f1eb_1f1f7.png"
  }
]


## Connect with mongo db

In [136]:
import json
from pymongo import MongoClient
from pymongo.errors import BulkWriteError
import os

# Configuration
MONGO_URI = "mongodb://localhost:27017/"  # À adapter selon votre configuration
DB_NAME = "chirac_db"
COLLECTION_NAME = "publications"
JSON_FILE = "C:/Users/yomna/Downloads/scrapper/Pictures/publications.json"

def get_mongo_collection():
    """Établit la connexion à MongoDB"""
    client = MongoClient(MONGO_URI)
    db = client[DB_NAME]
    return db[COLLECTION_NAME]

def import_json_to_mongodb():
    # 1. Vérifier que le fichier existe
    if not os.path.exists(JSON_FILE):
        print(f"Erreur: Le fichier {JSON_FILE} n'existe pas")
        return

    # 2. Lire le fichier JSON
    with open(JSON_FILE, 'r', encoding='utf-8') as f:
        try:
            publications = json.load(f)
        except json.JSONDecodeError as e:
            print(f"Erreur de lecture du JSON: {e}")
            return

    # 3. Connexion à MongoDB
    collection = get_mongo_collection()
    
    # 4. Nettoyage des données avant insertion
    processed_data = []
    for pub in publications:
        # Normalisation des champs
        processed = {
            'text': pub.get('text', '').strip(),
            'image': pub.get('image'),
            'source': 'facebook',  # Ajout d'un champ supplémentaire
            'import_date': datetime.datetime.now()
        }
        
        # Filtrage des publications vides
        if processed['text']:
            processed_data.append(processed)

    # 5. Insertion en masse avec gestion des erreurs
    if processed_data:
        try:
            result = collection.insert_many(processed_data, ordered=False)
            print(f"✅ {len(result.inserted_ids)} documents insérés avec succès")
            
            # Création d'un index sur le texte pour éviter les doublons
            collection.create_index([('text', 'text')])
            
        except BulkWriteError as e:
            print(f"⚠ Certains documents n'ont pas pu être insérés:")
            for err in e.details['writeErrors']:
                print(f"- Document {err['index']}: {err['errmsg']}")
            
            # Compter les documents insérés malgré les erreurs
            success_count = len(processed_data) - len(e.details['writeErrors'])
            print(f"{success_count} documents ont tout de même été insérés")

    else:
        print("Aucune donnée valide à importer")

if __name__ == "__main__":
    import datetime  # Pour le champ import_date
    import_json_to_mongodb()

✅ 15 documents insérés avec succès
